In [1]:
!pip install faiss-gpu

In [2]:
!pip3 install langchain
!pip3 install llama_index
!pip3 install openai
!pip3 install transformers
!pip3 install torch
!pip3 install sentence_transformers
!pip3 install pinecone-client
!pip3 install unstructured
!pip3 install openai
!pip3 install chromadb
! pip install typing-inspect==0.8.0 typing_extensions==4.5.0
! pip install pydantic==1.10.8

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [8]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR-OPEN_API_KEY'

In [9]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain,VectorDBQA, VectorDBQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [10]:
documents = UnstructuredFileLoader(
  '/content/Family_Care_by_W.W.Jacobs.pdf',
  mode='paged'
)
large_doc = documents.load()

[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading taggers: Package 'taggers' not found in
[nltk_data]     index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading taggers: Package 'taggers' not found in
[nltk_data]     index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nl

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.split_documents(large_doc)

In [13]:
texts[16].page_content

'"I shall ask her," said Mr. Barrett, doggedly. "I was going to wait a bit longer, but if there\'s any chance of her wrecking her prospects for life by marrying that tailor\'s dummy it\'s my duty to risk it—for her sake. I\'ve seen him talking to her twice myself, but I never thought he\'d dream of such a thing."'

In [14]:
from langchain.vectorstores import FAISS
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = FAISS.from_documents(texts, embeddings)

In [15]:
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:187: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:782: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:245: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [20]:
initial_prompt1 = """
You are analyzer of PDF file and you have to give answer based on data you have available
"""


In [21]:
query1 = """
who is author of book?
"""

In [22]:
result1 = qa({"query": query1, "prompt": initial_prompt1})

In [23]:
print(result1['result'])

The author of the book is W. W. Jacobs.


In [54]:
query2 = """
What are different characters are there described into books?
   """
result2 = qa({"query": query2, "prompt": initial_prompt1})

In [55]:
print(result2['result'])

The different characters described in the book are Mr. Barrett, Miss Lindsay, Charlotte (Mr. Barrett's sister-in-law), Jernshaw, and Louisa.


# VECTORDBAWITHSOurceschain


In [47]:
chain2 = VectorDBQAWithSourcesChain.from_llm(
  llm=ChatOpenAI(model='gpt-3.5-turbo-16k',temperature=0),
  vectorstore=docsearch, return_source_documents=True
)

In [52]:
query3 = """
What is the main concern of Mr. Barrett in this passage?
   """
result3 = chain2({"question": query3})

In [53]:
result3

{'question': '\nWhat is the main concern of Mr. Barrett in this passage?\n   ',
 'answer': "The main concern of Mr. Barrett in this passage is his matrimonial complications and the potential negative impact it may have on his daughter's prospects in life if she were to marry someone he disapproves of.\n",
 'sources': '/content/Family_Care_by_W.W.Jacobs.pdf',
 'source_documents': [Document(page_content='"I have no doubt it is much the best thing for the children to remain with their mother," she said, rising.\n\n"Much the best," agreed Mr. Barrett. "Whatever she is like," continued the old lady. "Are you ready, Louisa?"\n\nMr. Barrett followed them to the door, and then, returning to the room, watched, with glad eyes, their progress up the street.\n\n"Wonder whether she\'ll keep it to herself?" he muttered.\n\nHis doubts were set at rest next day. All Ramsbury knew by then of his matrimonial complications, and seemed anxious to talk about them; complications which tended to increase unt